![alt text](https://zewailcity.edu.eg/main/images/logo3.png)

_Prepared by_  [**Muhammad Hamdy AlAref**](mailto:malaref@zewailcity.edu.eg)

# Constraint Satisfaction Problems (CSPs)

All the search algorithms we discussed so far treat states as black boxes. However, we can gain a lot by using a *factored representation* instead! That is, we divide the state into a group of variables, their legal values, called the *domain*, and some constrains on those variable. A state is an *assignment* for some variables to values in the domain. An assignment is called *complete* if it includes *all* the variables and called *consistent* if it does not violate any of the constrains. A *solution* is a complete and consistent assignment.

Problems formulated in this way are called *Constraint Satisfaction Problems (CSPs)*.

## Problem Formulation

Following is a simple CSP formulation derived directly from the definition!

In [1]:
class CSP:
    '''
    Abstract base class for constraint satisfaction problem (CSP) formulation.
    It declares the expected methods to be used by a CSP search algorithm.
    All the methods declared are just placeholders that throw errors if not overridden by child "concrete" classes!
    '''
    
    def __init__(self):
        '''Constructor that initializes the problem. Typically used to setup the variables domains.'''
        self.domains = None  # A list of all the variables domains
                             # domains[0] is the set of values for the first variable
                             # domains[1] is the set of values for the second variable
                             # and so on...      
    
    def consistent(self, assignment, variable, value):
        '''Returns whether or not the given new variable/value pair is consistent with an already consistent assignment.'''
        raise NotImplementedError
    
    def related_variables(self, variable):
        '''Returns all the variables sharing a constraint with a variable. Only needed for inference.'''
        raise NotImplementedError

## Backtracking

Backtracking is a general depth-first search for solving CSPs. It finds solutions by assigning a value to one variable at a time and *backtracks* when no legal values are left for some variable to try alternative values for the previous assignments.

In [2]:
def backtracking(csp, verbose=False):
    '''Backtracking search implementation.'''
    if verbose: visualizer = Visualizer(csp)
    def backtrack(assignment):  # The recursive backtracking function with a partial assignment; assumes the assignment is always consistent
        if verbose: visualizer.visualize([assignment])
        if len(assignment) is len(csp.domains):  # Terminal condition: If the assignment is complete (all variables are assigned)
            return assignment  # Then, the assignment is complete; return it and you're done!
        variable = len(assignment)  # The next unassigned variable index
        for value in csp.domains[variable]:  # Iterating through all possible values in the variable's domain
            if csp.consistent(assignment, variable, value):  # If the new variable/value pair is consistent with the (already consistent) assignment
                assignment[variable] = value  # Add the new variable/value pair to assignment (the resulting assignment is guaranteed to be consistent)
                result = backtrack(assignment)  # Use the new resulting assignment to recursively call the function to try the next variable (if any)
                if result is not None:  # If the function returns with a consistent and complete assignment (solution)
                    return result  # Then, return this assignment (solution)
                del assignment[variable]  # Otherwise, remove the new variable/value pair and try another value
        return None  # If all the values fail, then there is no solution with this partial assignment
    return backtrack({})  # Start with an empty assignment (always consistent)

## Visualization

As always, we have some code to help us visualize the problem! We will be using our good-old visualizer!

In [3]:
from shutil import get_terminal_size
terminal_width, _ = get_terminal_size()

_visualizers = {}

def _default_visualizer(_, state):
    '''Generic visualizer for unknown problems.'''
    print(state)

class Visualizer:
    '''Visualization and printing functionality encapsulation.'''

    def __init__(self, problem):
        '''Constructor with the problem to visualize.'''
        self.problem = problem
        self.counter = 0
    
    def visualize(self, frontier):
        '''Visualizes the frontier at every step.'''
        self.counter += 1
        print(f'Frontier at step {self.counter}')
        for state in frontier:
            print()
            _visualizers.get(type(self.problem), _default_visualizer)(self.problem, state)
        print('-' * terminal_width)

def _sudoku_visualizer(problem, state):
    '''Custom visualizer for the sudoku puzzle CSP.'''
    for i in range(9):
        for j in range(3):
            for k in range(3):
                print(state.get(9 * i + 3 * j + k, 0), end=' ')
            if j < 2: print('|', end=' ')
        print()
        if i % 3 is 2 and i < 8: print('------+-------+------')

## Example Problem: Sudoku Puzzle

To illustrate the backtracking algorithm, we will use the problem that, unknowingly, introduced all newspaper readers to CSPs; the [Sudoku puzzle](https://en.wikipedia.org/wiki/Sudoku).

In [4]:
class Sudoku(CSP):
    '''Sudoku Puzzle CSP formulation.'''

    def __init__(self, partial=None):
        self.domains = []
        for i in range(81):  # We have 81 variables in Sudoku
            if partial is None or int(partial[i]) is 0:  # If there are no initial constraints on this variables (or no initial constraints are given at all)
                self.domains.append(set(range(1, 10)))  # Then, add all the possible values to its domain (1 to 9 inclusive)
            else:
                self.domains.append({int(partial[i])})  # Otherwise, make its initial value its one and only possible value
    
    def related_variables(self, variable):
        row = variable // 9  # The row of the variable
        col = variable % 9  # The column of the variable
        anchor = 27 * (variable // 27) + 3 * (col // 3)  # The top left corner of the box of the variable
        related_variables_list = [] # A list to hold all related variables
        related_variables_list.extend(range(col, 81, 9))  # Add all variables in the same column
        related_variables_list.extend(range(row * 9, (row + 1) * 9, 1))  # Add all variables in the same row
        for i in range(3):  # Iterating through all rows in a box
            related_variables_list.extend(range(anchor + i * 9, anchor + i * 9 + 3, 1))  # Add all variables in that row of the box
        return related_variables_list
    
    def consistent(self, assignment, variable, value):
        for i in self.related_variables(variable):  # Iterating through all related variables
            if assignment.get(i) is value:  # If a related variable has the same value
                return False  # Then, the new variable/value pair is not consistent with the (already consistent) assignment
        return True  # If execution reaches this point, then no related variable has the same value; that is, the new variable/value pair is consistent with the (already consistent) assignment

_visualizers[Sudoku] = _sudoku_visualizer

Let's try solving the Sudoku puzzle with backtracking search!

In [5]:
csp = Sudoku()
backtracking(csp, verbose=True)

Frontier at step 1

0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
--------------------------------------------------------------------------------
Frontier at step 2

1 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
--------------------------------------------------------------------------------
Frontier at step 3

1 2 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
-------------------------

1 2 3 | 4 5 6 | 7 8 9 
4 5 6 | 1 7 8 | 2 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
--------------------------------------------------------------------------------
Frontier at step 38

1 2 3 | 4 5 6 | 7 8 9 
4 5 6 | 1 7 8 | 2 3 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
--------------------------------------------------------------------------------
Frontier at step 39

1 2 3 | 4 5 6 | 7 8 9 
4 5 6 | 1 7 8 | 3 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
-------------------------------------------

0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
--------------------------------------------------------------------------------
Frontier at step 73

1 2 3 | 4 5 6 | 7 8 9 
4 5 6 | 1 9 8 | 2 3 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
--------------------------------------------------------------------------------
Frontier at step 74

1 2 3 | 4 5 6 | 7 8 9 
4 5 6 | 1 9 8 | 3 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
--------------------------------------------------------------------------------
Frontier

0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
--------------------------------------------------------------------------------
Frontier at step 100

1 2 3 | 4 5 6 | 7 8 9 
4 5 6 | 2 7 8 | 1 3 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
--------------------------------------------------------------------------------
Frontier at step 101

1 2 3 | 4 5 6 | 7 8 9 
4 5 6 | 2 7 8 | 3 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
--------------------------------------------------------------------------------
Fronti

0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
--------------------------------------------------------------------------------
Frontier at step 140

1 2 3 | 4 5 6 | 7 8 9 
4 5 6 | 3 1 2 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
--------------------------------------------------------------------------------
Frontier at step 141

1 2 3 | 4 5 6 | 7 8 9 
4 5 6 | 3 1 7 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
--------------------------------------------------------------------------------
Frontier at step 142

1 2 3 | 4 5 6 | 7 8 9 
4 5 6 

...

0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
--------------------------------------------------------------------------------
Frontier at step 278

1 2 3 | 4 5 6 | 7 8 9 
4 5 6 | 7 8 9 | 1 2 3 
7 8 9 | 1 2 3 | 4 5 6 
------+-------+------
2 1 4 | 3 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
--------------------------------------------------------------------------------
Frontier at step 279

1 2 3 | 4 5 6 | 7 8 9 
4 5 6 | 7 8 9 | 1 2 3 
7 8 9 | 1 2 3 | 4 5 6 
------+-------+------
2 1 4 | 3 6 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
--------------------------------------------------------------------------------
Frontier at step 280

1 2 3 | 4 5 6 | 7 8 9 
4 5 6 | 7 8 9 | 1 2 3 
7 8 9 | 1 2 3 | 4 5 6 
------+-------+------
2 1 4 | 3 6 5 | 0 0 0 
0 0 0 

0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
--------------------------------------------------------------------------------
Frontier at step 316

1 2 3 | 4 5 6 | 7 8 9 
4 5 6 | 7 8 9 | 1 2 3 
7 8 9 | 1 2 3 | 4 5 6 
------+-------+------
2 1 4 | 3 6 5 | 8 9 7 
3 6 5 | 8 4 1 | 2 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
--------------------------------------------------------------------------------
Frontier at step 317

1 2 3 | 4 5 6 | 7 8 9 
4 5 6 | 7 8 9 | 1 2 3 
7 8 9 | 1 2 3 | 4 5 6 
------+-------+------
2 1 4 | 3 6 5 | 8 9 7 
3 6 5 | 8 4 2 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
--------------------------------------------------------------------------------
Frontier at step 318

1 2 3 | 4 5 6 | 7 8 9 
4 5 6 | 7 8 9 | 1 2 3 
7 8 9 | 1 2 3 | 4 5 6 
------+-------+------
2 1 4 | 3 6 5 | 8 9 7 
3 6 5 

------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
--------------------------------------------------------------------------------
Frontier at step 350

1 2 3 | 4 5 6 | 7 8 9 
4 5 6 | 7 8 9 | 1 2 3 
7 8 9 | 1 2 3 | 4 5 6 
------+-------+------
2 1 4 | 3 6 5 | 8 9 7 
3 6 5 | 8 9 7 | 2 1 4 
8 9 7 | 2 1 4 | 3 6 5 
------+-------+------
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
--------------------------------------------------------------------------------
Frontier at step 351

1 2 3 | 4 5 6 | 7 8 9 
4 5 6 | 7 8 9 | 1 2 3 
7 8 9 | 1 2 3 | 4 5 6 
------+-------+------
2 1 4 | 3 6 5 | 8 9 7 
3 6 5 | 8 9 7 | 2 1 4 
8 9 7 | 2 1 4 | 3 6 5 
------+-------+------
5 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 0 0 
--------------------------------------------------------------------------------
Frontier at step 352

1 2 3 | 4 5 6 | 7 8 9 
4 5 6 | 7 8 9 | 1 2 3 
7 8 9 | 1 2 3 | 4 5 6 
------+-------+------
2 1 4 |

6 4 2 | 9 7 8 | 5 3 1 
9 7 8 | 5 3 0 | 0 0 0 
--------------------------------------------------------------------------------
Frontier at step 389

1 2 3 | 4 5 6 | 7 8 9 
4 5 6 | 7 8 9 | 1 2 3 
7 8 9 | 1 2 3 | 4 5 6 
------+-------+------
2 1 4 | 3 6 5 | 8 9 7 
3 6 5 | 8 9 7 | 2 1 4 
8 9 7 | 2 1 4 | 3 6 5 
------+-------+------
5 3 1 | 6 4 2 | 9 7 8 
6 4 2 | 9 7 8 | 5 3 1 
9 7 8 | 5 3 1 | 0 0 0 
--------------------------------------------------------------------------------
Frontier at step 390

1 2 3 | 4 5 6 | 7 8 9 
4 5 6 | 7 8 9 | 1 2 3 
7 8 9 | 1 2 3 | 4 5 6 
------+-------+------
2 1 4 | 3 6 5 | 8 9 7 
3 6 5 | 8 9 7 | 2 1 4 
8 9 7 | 2 1 4 | 3 6 5 
------+-------+------
5 3 1 | 6 4 2 | 9 7 8 
6 4 2 | 9 7 8 | 5 3 1 
9 7 8 | 5 3 1 | 6 0 0 
--------------------------------------------------------------------------------
Frontier at step 391

1 2 3 | 4 5 6 | 7 8 9 
4 5 6 | 7 8 9 | 1 2 3 
7 8 9 | 1 2 3 | 4 5 6 
------+-------+------
2 1 4 | 3 6 5 | 8 9 7 
3 6 5 | 8 9 7 | 2 1 4 
8 9 7 

{0: 1,
 1: 2,
 2: 3,
 3: 4,
 4: 5,
 5: 6,
 6: 7,
 7: 8,
 8: 9,
 9: 4,
 10: 5,
 11: 6,
 12: 7,
 13: 8,
 14: 9,
 15: 1,
 16: 2,
 17: 3,
 18: 7,
 19: 8,
 20: 9,
 21: 1,
 22: 2,
 23: 3,
 24: 4,
 25: 5,
 26: 6,
 27: 2,
 28: 1,
 29: 4,
 30: 3,
 31: 6,
 32: 5,
 33: 8,
 34: 9,
 35: 7,
 36: 3,
 37: 6,
 38: 5,
 39: 8,
 40: 9,
 41: 7,
 42: 2,
 43: 1,
 44: 4,
 45: 8,
 46: 9,
 47: 7,
 48: 2,
 49: 1,
 50: 4,
 51: 3,
 52: 6,
 53: 5,
 54: 5,
 55: 3,
 56: 1,
 57: 6,
 58: 4,
 59: 2,
 60: 9,
 61: 7,
 62: 8,
 63: 6,
 64: 4,
 65: 2,
 66: 9,
 67: 7,
 68: 8,
 69: 5,
 70: 3,
 71: 1,
 72: 9,
 73: 7,
 74: 8,
 75: 5,
 76: 3,
 77: 1,
 78: 6,
 79: 4,
 80: 2}

A rather difficult instance?

In [ ]:
csp = Sudoku('\
530070000\
600195000\
098000060\
800060003\
400803001\
700020006\
060000280\
000419005\
000080079')

%timeit backtracking(csp)

## Speeding-up Backtracking

One of the benefits of CSP formulation is that useful heuristics can be derived irrespective of the problem itself! We are going to look at one famous heuristic for each of the types we mentioned.

For variable ordering, a famous one is the *minimum-remaining-values (MRV)* heuristic. It chooses the variable with the minimum legal values left in its domain (given the problem constraints). This mirrors what humans do when solving CSPs by hand, e.g. newspaper Sudoku problems. Another useful one is the *degree* heuristic. It chooses the variable that enforces the largest number of constraints on other unassigned variables. In practice, this serves as a good tie breaker for MRV, especially in the first steps.

For interweaving inferences, *forward-checking* is a very simple one. It establishes *arc consistency* for a variable *whenever we assign it* by removing illegal values from all the domains of the variables involved with it in a constraint. This is especially helpful with aforementioned MRV heuristic since it basically provides the information MRV needs to work!

A more complicated inference algorithm is called *AC-3*. Unlike forward-checking, it establishes arc consistency for *all* the variables instead of one variable at a time. Using (a variant of) AC-3 interwoven with backtracking yields an algorithm called *Maintaining Arc Consistency (MAC)*.

Here are implementations for the MRV and the forward-checking heuristics!

In [ ]:
from math import inf
from copy import deepcopy

def select_unassigned_variable(domains):  # A straightforward implementation of the MRV heuristic
    '''Returns the next unassigned variable to be considered.'''
    variable = None  # The variable to be considered next
    minimum = inf  # Initiate the minimum number of values in a variable's domain to infinity (since any variable will have less values than that)
    for i, domain in enumerate(domains):  # Iterating through all the variables domains (remaining values)
        if domain is not None and len(domain) < minimum:  # If an unassigned variable's number of remaining values is less the minimum seen so far
            variable = i  # Then, set this variable as the one to be considered next
            minimum = len(domain)  # And set its number of remaining values as the minimum seen so far
    return variable

def inference(csp, variable, value, domains):  # A straightforward implementation of forward checking inference (arc consistency)
    '''Returns inferences and filtered domains based on the new variable/value pair.'''
    inferences = {}
    filtered_domains = deepcopy(domains)  # Initialize the new filtered domains are the given ones
    filtered_domains[variable] = None  # Remove the domain of the given variable (mark it as assigned)
    for related_variable in csp.related_variables(variable):  # Iterating through all related variables
        if filtered_domains[related_variable]:  # If the related variable's domain is not yet removed (it is not yet assigned)
            filtered_domains[related_variable].discard(value)  # Remove the given value from the related variable's domain
            if len(filtered_domains[related_variable]) is 0:  # If the related variable's domain becomes empty
                return None  # Then, this new variable/value pair will cause an inconsistency later; better stop now
            if len(filtered_domains[related_variable]) is 1:  # If the related variable's domain becomes narrowed down to ONE value
                inferences[related_variable] = filtered_domains[related_variable].pop()  # Then, this related variable is "inferred" to be assigned to this value
                filtered_domains[related_variable] = None  # Remove the inferred related variable's domain
    return inferences, filtered_domains


Backtracking *works* but we can speed it up with heuristics to *guide* it just like informed search! We are going to discuss two types of heuristics; *variables ordering* and *interweaving inferences*. (Inferences can be used as pre-processors before running backtracking. However, they can be really useful when run before each step as well.) There are other heuristics to be included that you are encouraged to learn about, e.g., *values ordering*, *intelligent backtracking*, etc., but they are out of the scope of this notebook.

We can then modify the backtracking algorithm to make use of such heuristics!

In [ ]:
def backtracking(csp, verbose=False):
    '''Backtracking search implementation.'''
    if verbose: visualizer = Visualizer(csp)
    def backtrack(assignment, domains):  # The recursive backtracking function with a partial assignment and current variables domains; assumes the assignment is always consistent
        if verbose: visualizer.visualize([assignment])
        if len(assignment) is len(csp.domains):  # Terminal condition: If the assignment is complete (all variables are assigned)
            return assignment  # Then, the assignment is complete; return it and you're done!
        var = select_unassigned_variable(domains)  # Choose the next unassigned variable to be considered
        for value in domains[var]:  # Iterating through all possible values in the variable's CURRENT domain
            if csp.consistent(assignment, var, value):  # If the new variable/value pair is consistent with the (already consistent) assignment
                inferences, filtered_domains = inference(csp, var, value, domains)  # Based on this new variable/value pair, infer the new domains of coming variables (and possibly their exact (corollary) assignments if their domains are narrowed down to one possible value)
                if inferences is None:  # If an inconsistency is detected during inference
                    continue  # Then, no need to go further; this value is no good
                assignment[var] = value  # Add the new variable/value pair to assignment (the resulting assignment is guaranteed to be consistent)
                assignment.update(inferences)  # Add the extra inferences to assignment (the resulting assignment is guaranteed to be consistent)
                result = backtrack(assignment, filtered_domains)  # Use the new resulting assignment and domains to recursively call the function to try the next variable (if any)
                if result is not None:  # If the function returns with a consistent and complete assignment (solution)
                    return result  # Then, return this assignment (solution)
                del assignment[var]  # Otherwise, remove the new variable/value pair and try another value
                for var in inferences:  # Also, remove the extra inferences based on that new variable/value pair
                    del assignment[var]
        return None
    return backtrack({}, csp.domains)

Let's try solving the Sudoku puzzle with improved backtracking search!

In [ ]:
csp = Sudoku()
backtracking(csp, verbose=True)

Now, let's try the new implementation with the difficult instance!

In [ ]:
csp = Sudoku('\
530070000\
600195000\
098000060\
800060003\
400803001\
700020006\
060000280\
000419005\
000080079')

%timeit backtracking(csp)

## Requirement

Let's try solving another famous CSP puzzle; the [Zebra Puzzle](https://en.wikipedia.org/wiki/Zebra_Puzzle) (aka, Einstein's Riddle)!

Following is the puzzle description as appeared in *Life International* in 1962:

* There are five houses.
* The Englishman lives in the red house.
* The Spaniard owns the dog.
* Coffee is drunk in the green house.
* The Ukrainian drinks tea.
* The green house is immediately to the right of the ivory house.
* The Old Gold smoker owns snails.
* Kools are smoked in the yellow house.
* Milk is drunk in the middle house.
* The Norwegian lives in the first house.
* The man who smokes Chesterfields lives in the house next to the man with the fox.
* Kools are smoked in the house next to the house where the horse is kept.
* The Lucky Strike smoker drinks orange juice.
* The Japanese smokes Parliaments.
* The Norwegian lives next to the blue house.

**Who drinks water? Who owns the zebra?**

You are required to write Python code that implements this riddle and use backtracking search to answer the question!

**Estimated time for this exercise is 30 minutes!**

## _Optional_ Pythonic Implementations

Following are compact, so-called _pythonic_, re-implementations of some of the codes above. These are completely optional and intended for readers interested in advanced Python shortcuts and optimizations.

**DO NOT WASTE TIME UNDERSTANDING IT TILL YOU ARE DONE WITH THE REQUIREMENT!**

In [0]:
from itertools import chain

class Sudoku(CSP):
    '''Sudoku Puzzle CSP formulation.'''

    def __init__(self, partial=None):
        self.domains = [set(range(1, 10)) if partial is None or int(partial[i]) is 0 else {int(partial[i])} for i in range(81)]
    
    def related_variables(self, variable):
        '''Helper method for the Sudoku puzzle to get all the variables in the same row, column or box as the given variable.'''
        row = variable // 9
        col = variable % 9
        anchor = 27 * (variable // 27) + 3 * (col // 3)
        return chain(range(row * 9, (row + 1) * 9, 1),
                     range(col, 81, 9),
                     *(range(anchor + i * 9, anchor + i * 9 + 3) for i in range(3)))
    
    def consistent(self, assignment, variable, value):
        return not any(assignment.get(i, None) is value for i in self.related_variables(variable))


def _sudoku_visualizer(problem, state):
    '''Custom visualizer for the sudoku puzzle CSP.'''
    for i in range(9):
        for j in range(3):
            for k in range(3):
                print(state.get(9 * i + 3 * j + k, 0), end=' ')
            if j < 2: print('|', end=' ')
        print()
        if i % 3 is 2 and i < 9: print('------+-------+------')

_visualizers[Sudoku] = _sudoku_visualizer

In [0]:
from math import inf

def select_unassigned_variable(domains):
    return min(range(len(domains)), key=lambda index: len(domains[index]) if domains[index] else inf)